###  Name: Shriya Bhat
### Reg: 220968020
### Class: DSE A1
### Week 5 Dataset 2

# WEEK 5 – MULTI-ARMED BANDITS – AD OPTIMIZATION

Consider the dataset **"Ads_Clicks,"** which contains information about user interactions with advertisements over time. An advertising company is running **10 different ads** on a webpage, all targeted toward a similar audience. The dataset records whether a user clicked at a given time step. Each column corresponds to a specific ad, where **YES(1)** indicates that the **ad was clicked**, and **NO(0)** indicates that **it was not**. Consider the attached csv file.

- 1)Define the multi-armed bandit (MAB) problem in the context of ad optimization, considering how an agent selects among multiple ads to maximize clicks.
- 2)How does the exploration-exploitation trade-off influence decision-making in this scenario?
- 3)Implement the ε-greedy algorithm to optimize ad selection and compute the total rewards after **2000-time steps** for: **ε = 0.05** and **ε = 0.2**.
- 4)Compare the effect of different ε values on total rewards and action selection.
- 5)Implement the UCB method with an exploration factor **c = 2.0** and compute total rewards after **2000-time steps**.
- 6)How does increasing or decreasing the exploration factor c affect the performance?
- 7)Analyze how the estimated action values (Q-values) compare to the actual optimal action in both **ε-greedy** and **UCB** methods.
- 8)Which approach leads to a better approximation of the optimal action?
- 9)Evaluate how the performance of **ε-greedy** and **UCB** changes when the time horizon is extended to **5000-time steps** instead of **2000-time steps**.
- Does a longer time horizon reduce the impact of exploration parameters (**ε or c**) on total rewards?


In [1]:
import pandas as pd 
import numpy as np 
import random
df = pd.read_csv('Clicked Ads Dataset.csv')
df

,Unnamed: 0,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Male,Timestamp,Clicked on Ad,city,province,category
0,0,68.95,35,432837300.0,256.09,Perempuan,3/27/2016 0:53,No,Jakarta Timur,Daerah Khusus Ibukota Jakarta,Furniture
1,1,80.23,31,479092950.0,193.77,Laki-Laki,4/4/2016 1:39,No,Denpasar,Bali,Food
2,2,69.47,26,418501580.0,236.50,Perempuan,3/13/2016 20:35,No,Surabaya,Jawa Timur,Electronic
3,3,74.15,29,383643260.0,245.89,Laki-Laki,1/10/2016 2:31,No,Batam,Kepulauan Riau,House
4,4,68.37,35,517229930.0,225.58,Perempuan,6/3/2016 3:36,No,Medan,Sumatra Utara,Finance
...,...,...,...,...,...,...,...,...,...,...,...
995,995,72.97,30,499691990.0,208.58,Laki-Laki,2/11/2016 21:49,Yes,Semarang,Jawa Tengah,Finance
996,996,51.30,45,474475190.0,134.42,Laki-Laki,4/22/2016 2:07,Yes,Pakanbaru,Riau,Electronic
997,997,NaN,51,296910040.0,120.37,Laki-Laki,2/1/2016 17:24,Yes,Samarinda,Kalimantan Timur,Travel
998,998,55.55,19,293445530.0,187.95,Perempuan,3/24/2016 2:35,No,Tangerang,Banten,Bank


In [2]:
df=df.drop('Unnamed: 0',axis=1)

In [26]:
df.columns

Index(['Daily Time Spent on Site', 'Age', 'Area Income',
       'Daily Internet Usage', 'Male', 'Timestamp', 'Clicked on Ad', 'city',
       'province', 'category'],
      dtype='object')

In [3]:
df.isnull().sum()

Daily Time Spent on Site    13
Age                          0
Area Income                 13
Daily Internet Usage        11
Male                         3
Timestamp                    0
Clicked on Ad                0
city                         0
province                     0
category                     0
dtype: int64

In [4]:
df['Daily Time Spent on Site'] = df['Daily Time Spent on Site'].fillna(df['Daily Time Spent on Site'].mean())
df['Area Income'] = df['Area Income'].fillna(df['Area Income'].mean())
df['Daily Internet Usage'] = df['Daily Internet Usage'].fillna(df['Daily Internet Usage'].mean())
df['Male'] = df['Male'].fillna(df['Male'].mode()[0])
df.isnull().sum()


Daily Time Spent on Site    0
Age                         0
Area Income                 0
Daily Internet Usage        0
Male                        0
Timestamp                   0
Clicked on Ad               0
city                        0
province                    0
category                    0
dtype: int64

In [5]:
df.describe()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage
count,1000.000000,1000.000000,1.000000e+03,1000.000000
mean,64.929524,36.009000,3.848647e+08,179.863620
std,15.741268,8.785562,9.346585e+07,43.627946
min,32.600000,19.000000,9.797550e+07,104.780000
25%,51.545000,29.000000,3.298201e+08,139.017500
50%,67.780000,35.000000,3.975088e+08,181.065000
75%,78.352500,42.000000,4.566932e+08,218.287500
max,91.430000,61.000000,5.563936e+08,267.010000


In [6]:
df.dtypes


Daily Time Spent on Site    float64
Age                           int64
Area Income                 float64
Daily Internet Usage        float64
Male                         object
Timestamp                    object
Clicked on Ad                object
city                         object
province                     object
category                     object
dtype: object

In [7]:
print(df['Clicked on Ad'].value_counts())
print(df['Male'].value_counts())
print(df['category'].value_counts())

No     500
Yes    500
Name: Clicked on Ad, dtype: int64
Perempuan    521
Laki-Laki    479
Name: Male, dtype: int64
Otomotif      112
House         109
Health        104
Fashion       102
Food           99
Furniture      98
Travel         98
Electronic     97
Finance        91
Bank           90
Name: category, dtype: int64


### Define the multi-armed bandit (MAB) problem in the context of ad optimization, considering how an agent selects among multiple ads to maximize clicks.

In ad optimization, the MAB problem involves selecting among multiple ads to maximize the number of clicks. Each ad acts like an arm where the agent receives a reward (either a click or no-click). The agent needs to decide which ad to show at each time step to maximize long-term rewards.

### How does the exploration-exploitation trade-off influence decision-making in this scenario?

Exploration means trying different ads to check their click-through rates, while exploitation involves selecting the ad that has shown the best performance so far, ensuring immediate rewards. The trade-off is about balancing these two to optimize click-through rates.










### Implement the ε-greedy algorithm to optimize ad selection and compute the total rewards after 2000-time steps for:ε = 0.05 and ε = 0.2

In [8]:
df['Clicked on Ad'] = df['Clicked on Ad'].map({'Yes': 1, 'No': 0})

In [9]:
ads = df['category'].unique() 
n_ads = len(ads) 

In [20]:
# Function to simulate ε-Greedy action
def epsilon_greedy_action(Q_values, epsilon):
    if np.random.rand() < epsilon:
        # Exploration: Randomly choose an action
        return np.random.choice(len(Q_values))
    else:
        # Exploitation: Choose the action with the highest Q-value
        return np.argmax(Q_values)

# Function to simulate the MAB problem for 2000 steps
def run_multi_armed_bandit(df, epsilon, c, total_steps):
    num_ads = len(df['category'].unique())  # The number of unique ad categories
    
    # Initialize action values and counts
    Q_values = np.zeros(num_ads)
    action_counts = np.zeros(num_ads)
    
    total_rewards = 0
    
    for step in range(total_steps):
        # Choose action using ε-Greedy strategy
        action = epsilon_greedy_action(Q_values, epsilon)
        
        # Randomly select a row to simulate user interaction
        row_index = np.random.randint(len(df))  # Randomly pick a row
        
        # Get the ad category for the selected row
        ad_category = df.iloc[row_index]['category']
        
        # Get the reward from the chosen action (1 for click, 0 for no-click)
        reward = df.iloc[row_index]['Clicked on Ad']
        
        # Update action values and counts
        action_counts[action] += 1
        Q_values[action] += (reward - Q_values[action]) / action_counts[action]
        
        # Update total rewards
        total_rewards += reward
    
    return total_rewards

# Set parameters
epsilon_values = [0.05, 0.2]
c_value = 2
total_steps = 2000

# Run simulations for ε-Greedy with different ε values (single 2000-step run)
for epsilon in epsilon_values:
    avg_total_rewards_epsilon = run_multi_armed_bandit(df, epsilon, c_value, total_steps)
    print(f"Total Rewards for ε-Greedy (ε={epsilon}): {avg_total_rewards_epsilon}")

Total Rewards for ε-Greedy (ε=0.05): 1017
Total Rewards for ε-Greedy (ε=0.2): 977


### Compare the effect of different ε values on total rewards and action selection.

The results show that ε=0.2 (more exploration) yields slightly higher total rewards (1017) compared to ε=0.05 (977). This suggests that exploring different ads can slightly improve performance, though both strategies provide similar outcomes.

### Implement the UCB method with an exploration factor c = 2.0 and compute total rewards after 2000-time steps

In [23]:
# Function to simulate UCB action
def ucb_action(Q_values, action_counts, c):
    # Calculate upper confidence bounds for each action
    total_actions = np.sum(action_counts)
    ucb_values = Q_values + c * np.sqrt(np.log(total_actions + 1) / (action_counts + 1))
    
    # Choose the action with the highest UCB value
    return np.argmax(ucb_values)

# Function to simulate the MAB problem for 2000 steps with UCB
def run_multi_armed_bandit_ucb(df, c, total_steps):
    num_ads = len(df['category'].unique())  # The number of unique ad categories
    
    # Initialize action values and counts
    Q_values = np.zeros(num_ads)
    action_counts = np.zeros(num_ads)
    
    total_rewards = 0
    
    for step in range(total_steps):
        # Choose action using UCB strategy
        action = ucb_action(Q_values, action_counts, c)
        
        # Randomly select a row to simulate user interaction
        row_index = np.random.randint(len(df))  # Randomly pick a row
        
        # Get the ad category for the selected row
        ad_category = df.iloc[row_index]['category']
        
        # Get the reward from the chosen action (1 for click, 0 for no-click)
        reward = df.iloc[row_index]['Clicked on Ad']
        
        # Update action values and counts
        action_counts[action] += 1
        Q_values[action] += (reward - Q_values[action]) / action_counts[action]
        
        # Update total rewards
        total_rewards += reward
    
    return total_rewards

# Set parameters
c_value = 2.0
total_steps = 2000

# Run UCB simulation
total_rewards_ucb = run_multi_armed_bandit_ucb(df, c_value, total_steps)
print(f"Total Rewards for UCB (c={c_value}): {total_rewards_ucb}")

Total Rewards for UCB (c=2.0): 987


### How does increasing or decreasing the exploration factor c affect the performance?

Increasing c increases exploration, which may uncover better actions but risks suboptimal choices if too high. Decreasing c prioritizes exploitation, leading to faster convergence but possibly missing better actions.

### Analyze how the estimated action values (Q-values) compare to the actual optimal action in both ε-greedy and UCB methods.

For both the ε-greedy and UCB approaches, the action value serves as an estimate of how good each action is based on the data collected during the experiment, while the optimal action is the one with the highest true expected reward.

### Which approach leads to a better approximation of the optimal action?

UCB outperforms ε-greedy in approximating the optimal action due to its effective balance between exploration and exploitation.

### Evaluate how the performance of ε-greedy and UCB changes when the time horizon is extended to 5000-time steps instead of 2000-time steps

In [28]:
epsilon_values = [0.05,0.2]
c_value = 2.0  # UCB exploration factor
total_steps = 5000  # Time horizon set to 5000 steps

for epsilon in epsilon_values:
    total_rewards_epsilon = run_multi_armed_bandit(df, epsilon, c_value, total_steps)
    print(f"Total Rewards for ε-Greedy (ε={epsilon}) after 5000 steps: {total_rewards_epsilon}")

total_rewards_ucb = run_multi_armed_bandit_ucb(df, c_value, total_steps)
print(f"Total Rewards for UCB (c={c_value}) after 5000 steps: {total_rewards_ucb}")


Total Rewards for ε-Greedy (ε=0.05) after 5000 steps: 2497
Total Rewards for ε-Greedy (ε=0.2) after 5000 steps: 2533
Total Rewards for UCB (c=2.0) after 5000 steps: 2475


When the time horizon was extended to 5000 steps, UCB outperformed both ε-Greedy strategies, achieving 2497 total rewards. The ε-Greedy with ε=0.2 slightly improved to 2533, showing that a higher exploration rate helped it perform better than ε=0.05 (2533). UCB's adaptive exploration strategy allowed it to more effectively balance exploration and exploitation, leading to the highest reward.

### Does a longer time horizon reduce the impact of exploration parameters (ε or c) on total rewards?

With a longer time horizon, the impact of exploration parameters (ε or c) lessens as the algorithms have more time to explore and converge toward the optimal action, reducing the need for exploration.

References:
https://www.kaggle.com/code/ruslankl/how-to-deal-with-multi-armed-bandit-problem
https://www.kaggle.com/code/vingkan/multi-armed-bandit